In [42]:
import requests
from hashlib import sha1
from utils import generate_random
from dotenv import Dotenv

Config

In [43]:
env = Dotenv(".env")
hostname = env["hostname"]
url = f"https://{hostname}/jsonrpc/"
username = env["username"]
password = env["password"]

In [44]:
ha1 = sha1(f"{username}:Insta-NetBox:{password}".encode()).digest().hex()
ha1u = ha1.upper()
ha1u

'D570F7C57ED8ECBA1335C7EA13D1A746AD96DA00'

In [45]:
ha2 = sha1("POST:/jsonrpc//management".encode()).digest().hex()
ha2u = ha2.upper()
ha2u

'D3178078CA6013FFFAC1C7AE1C25796EC830DCD3'

In [46]:
ncd = "1"
nc = ("00000000" + ncd)[-8:]
nc

'00000001'

In [47]:
cnonce = generate_random(40)
cnonce
cnonce = "ZhbBsyHKXCdXpu6zoEDH28LJazRnwOOGpZLpppyX"

In [48]:
res = requests.post(
    url + "management",
    json={
        "jsonrpc":"2.0",
        "method": "getDigestAuthentificationInfos",
        "params": None
        },
    verify=False
)
result = res.json()["result"]
isid = res.headers["x-clientcredentials-sessionid"]

c:\Users\MaikS\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'enetserver.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [49]:
ha3 = sha1(f"{ha1u}:{result['nonce']}:{nc}:{cnonce}:auth:{ha2u}".encode()).digest().hex()
response = ha3.upper()
response

'743816FBF016DE3B6B1DBFC9E8384A0F368BF06B'

In [50]:
auth_request_params = {
    "username": username,
    "realm": result["realm"],
    "nonce": result["nonce"],
    "uri": result["uri"],
    "qop": result["qop"],
    "opaque": result["opaque"],
    "response": response,
    "nc": nc,
    "cnonce": cnonce
}

In [51]:
payload = {
    "userName": username,
    "uri": "/jsonrpc//management",
    "qop": "auth",
    "cnonce": auth_request_params["cnonce"],
    "nc": auth_request_params["nc"],
    "response": response,
    "nonce": auth_request_params["nonce"],
    "algorithm": "sha",
    "opaque": auth_request_params["opaque"]
}

In [52]:
login_result = requests.post(
    url + "management",
        json={
        "jsonrpc": "2.0",
        "method": "userLoginDigest",
        "params": payload
        },
    headers={"Cookie": f"INSTASESSIONID={isid}"},
    verify=False
)
login_result.json()

c:\Users\MaikS\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'enetserver.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'jsonrpc': '2.0',
 'error': {'code': -32602, 'message': 'Invalid params'},
 'id': None}